In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [39]:
path = 'CIFAR10/'
cifar10 = torchvision.datasets.CIFAR10(
    path, train=True, download=True, transform=torchvision.transforms.ToTensor())
cifar10_val = torchvision.datasets.CIFAR10(
    path, train=False, download=True, transform=torchvision.transforms.ToTensor())


Files already downloaded and verified
Files already downloaded and verified


In [40]:
# imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
# print(imgs.view(3, -1).mean(dim=1))
# print(imgs.view(3, -1).std(dim=1))

In [41]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616])])


In [42]:
cifar10 = torchvision.datasets.CIFAR10(path, train=True, download=True, transform=transform)
cifar10_val = torchvision.datasets.CIFAR10(path, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [43]:
train_loader = torch.utils.data.DataLoader(
    cifar10, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    cifar10_val, batch_size=4, shuffle=False)


In [44]:
class Net(nn.Module):
    def __init__(self, n_chansl=32):
        super().__init__()
        self.conv1 = nn.Conv2d(3, n_chansl, 3, padding=1)
        self.conv2 = nn.Conv2d(n_chansl, 8, 3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)     
        out = torch.flatten(out,1)
        out = F.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
       

In [45]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            output = model(imgs)
            loss = loss_fn(output, labels)
           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        print('Epoch {}, Training loss {}'.format(epoch, loss_train / len(train_loader)))


In [46]:
print(device)

cuda:0


In [47]:
model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(100, optimizer, model, loss_fn, train_loader)


Epoch 1, Training loss 1.9313961649370193
Epoch 2, Training loss 1.5170884510350227
Epoch 3, Training loss 1.3638952237653732
Epoch 4, Training loss 1.2825539254432916
Epoch 5, Training loss 1.2222682726973295
Epoch 6, Training loss 1.1707663585448265
Epoch 7, Training loss 1.1248904776847362
Epoch 8, Training loss 1.083048212696314
Epoch 9, Training loss 1.0473564355558156
Epoch 10, Training loss 1.015079094157517
Epoch 11, Training loss 0.987778960993588
Epoch 12, Training loss 0.9665649177721143
Epoch 13, Training loss 0.9455823282226921
Epoch 14, Training loss 0.930029220340997
Epoch 15, Training loss 0.9151581468251347
Epoch 16, Training loss 0.9017299544516206
Epoch 17, Training loss 0.8908622128425538
Epoch 18, Training loss 0.8792849678631127
Epoch 19, Training loss 0.8678880205041171
Epoch 20, Training loss 0.8591948400755227
Epoch 21, Training loss 0.8487216512706875
Epoch 22, Training loss 0.8415119955779612
Epoch 23, Training loss 0.831757701061666
Epoch 24, Training loss 0

In [48]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0] 
                correct += int((predicted == labels).sum()) 

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.63
